In [1]:
# Import Library
import cv2
import mediapipe as mp


# Hitung Jarak Euclidean antara 2 titik landmark
def distance(p1, p2):
    return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2) ** 0.5

# Inisialisasi MediaPipe Face Mesh untuk deteksi wajah
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Inisialisasi MediaPipe Hands untuk deteksi tangan
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,  # cukup deteksi 1 tangan
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# Buka kamera / Webcam
cap = cv2.VideoCapture(0)

# Garis batas tengah (vertikal) frame 640x480 (ubah jika ukuran frame beda)
line_x = 320

# Variabel untuk hitung gerakan tangan
count_left_to_right = 0
count_right_to_left = 0
prev_x = None 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Mirror supaya seperti cermin
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # --------- Deteksi wajah & senyum -------------
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_results = face_mesh.process(rgb)

    smile_label = ""

    if face_results.multi_face_landmarks:
        for landmarks in face_results.multi_face_landmarks:
            # Landmark bibir penting
            left_lip = landmarks.landmark[61]
            right_lip = landmarks.landmark[291]
            top_lip = landmarks.landmark[13]
            bottom_lip = landmarks.landmark[14]

            # Cek sudut bibir terangkat
            left_corner = landmarks.landmark[61]
            right_corner = landmarks.landmark[291]

            # Hitung jarak bibir
            mouth_width = distance(left_lip, right_lip)
            mouth_height = distance(top_lip, bottom_lip)

            # Cek apakah sudut bibir terangkat (y lebih kecil artinya lebih tinggi)
            is_left_corner_up = left_corner.y < top_lip.y
            is_right_corner_up = right_corner.y < top_lip.y

            # Threshold (tuning bisa dilakukan)
            smile_width_threshold = 0.01
            mouth_height_threshold = 0.03

            if (mouth_width > smile_width_threshold and
                is_left_corner_up and is_right_corner_up and
                mouth_height < mouth_height_threshold):
                smile_label = "Senyum"
            else:
                smile_label = "Tidak Senyum"

            # Gambar landmark wajah
            mp_drawing.draw_landmarks(
                frame,
                landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,128,255), thickness=1)
            )

    # Deteksi tangan dan gerakan kiri <-> kanan
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb)

    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            # Gambar landmark tangan
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Landmark nomor 9 sebagai referensi tengah tangan (pergelangan tangan/ tengah telapak)
            cx = int(hand_landmarks.landmark[9].x * w)
            cy = int(hand_landmarks.landmark[9].y * h)

            # Gambar titik tengah tangan
            cv2.circle(frame, (cx, cy), 10, (0, 255, 255), -1)

            # Hitung crossing garis vertikal
            if prev_x is not None:
                if prev_x < line_x <= cx:
                    count_left_to_right += 1
                elif prev_x > line_x >= cx:
                    count_right_to_left += 1
            prev_x = cx

    # Gambar garis vertikal tengah
    cv2.line(frame, (line_x, 0), (line_x, h), (255, 0, 0), 2)

    if count_left_to_right >= 7 and count_right_to_left >= 7 and smile_label == "Senyum":
        cv2.putText(frame, 'Pintu Terbuka', (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    cv2.putText(frame, f"Kiri ke Kanan: {count_left_to_right}", (10, h - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    cv2.putText(frame, f"Kanan ke Kiri: {count_right_to_left}", (10, h - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
    cv2.putText(frame, f"{smile_label}", (10, h - 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
    # Tampilkan frame akhir
    cv2.imshow("Deteksi Senyum & Gerakan Tangan", frame)

    # Keluar dengan tombol 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan resource
cap.release()
cv2.destroyAllWindows()



I0000 00:00:1749112646.235443 6329771 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
I0000 00:00:1749112646.242617 6329771 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1749112646.252481 6342413 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749112646.262162 6342423 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749112646.270798 6342413 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749112646.275314 6342423 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t

KeyboardInterrupt: 

In [ ]:
# Import Library
import cv2
import mediapipe as mp


# Hitung Jarak Euclidean antara 2 titik landmark
def distance(p1, p2):
    return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2) ** 0.5

# Inisialisasi MediaPipe Face Mesh untuk deteksi wajah
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Inisialisasi MediaPipe Hands untuk deteksi tangan
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,  # cukup deteksi 1 tangan
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# Buka kamera / Webcam
cap = cv2.VideoCapture(0)

# Garis batas tengah (vertikal) frame 640x480 (ubah jika ukuran frame beda)
line_x = 320

# Variabel untuk hitung gerakan tangan
count_left_to_right = 0
prev_x = None 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Mirror supaya seperti cermin
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # --------- Deteksi wajah & senyum -------------
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_results = face_mesh.process(rgb)

    smile_label = ""

    if face_results.multi_face_landmarks:
        for landmarks in face_results.multi_face_landmarks:
            # Landmark bibir penting
            left_lip = landmarks.landmark[61]
            right_lip = landmarks.landmark[291]
            top_lip = landmarks.landmark[13]
            bottom_lip = landmarks.landmark[14]

            # Cek sudut bibir terangkat
            left_corner = landmarks.landmark[61]
            right_corner = landmarks.landmark[291]

            # Hitung jarak bibir
            mouth_width = distance(left_lip, right_lip)
            mouth_height = distance(top_lip, bottom_lip)

            # Cek apakah sudut bibir terangkat (y lebih kecil artinya lebih tinggi)
            is_left_corner_up = left_corner.y < top_lip.y
            is_right_corner_up = right_corner.y < top_lip.y

            # Threshold 
            smile_width_threshold = 0.01
            mouth_height_threshold = 0.03

            if (mouth_width > smile_width_threshold and
                is_left_corner_up and is_right_corner_up and
                mouth_height < mouth_height_threshold):
                smile_label = "Senyum"
            else:
                smile_label = "Tidak Senyum"

            # Gambar landmark wajah
            mp_drawing.draw_landmarks(
                frame,
                landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,128,255), thickness=1)
            )

    # Deteksi tangan dan gerakan kiri <-> kanan
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb)

    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            # Gambar landmark tangan
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Landmark nomor 9 sebagai referensi tengah tangan (pergelangan tangan/ tengah telapak)
            cx = int(hand_landmarks.landmark[9].x * w)
            cy = int(hand_landmarks.landmark[9].y * h)

            # Gambar titik tengah tangan
            cv2.circle(frame, (cx, cy), 10, (0, 255, 255), -1)

            # Hitung crossing garis vertikal
            if prev_x is not None:
                if prev_x < line_x <= cx:
                    count_left_to_right += 1
            prev_x = cx

    # Gambar garis vertikal tengah
    cv2.line(frame, (line_x, 0), (line_x, h), (255, 0, 0), 2)

    if count_left_to_right >= 7 and smile_label == "Senyum":
        cv2.putText(frame, 'Pintu Terbuka', (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    cv2.putText(frame, f"Kiri ke Kanan: {count_left_to_right}", (10, h - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    cv2.putText(frame, f"{smile_label}", (10, h - 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
    # Tampilkan frame akhir
    cv2.imshow("Deteksi Senyum & Gerakan Tangan Untuk Membuka Pintu", frame)

    # Keluar dengan tombol 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan resource
cap.release()
cv2.destroyAllWindows()



I0000 00:00:1749113062.325632 6349736 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1749113062.345280 6352842 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1749113062.349083 6349736 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1749113062.426794 6352842 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749113062.506879 6352857 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749113062.588140 6352857 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t